# [Memory](https://langchain-ai.github.io/langgraph/concepts/memory/)
- LangGraph의 Memory는 AI가 대화 내용을 기억할 수 있게 해주는 기능입니다. 
- 사람이 대화할 때 이전에 말한 내용을 기억하는 것처럼, AI도 과거 대화를 기억해서 더 자연스러운 대화를 할 수 있게 해줍니다.


## 왜 Memory가 필요한가요?
> Memory가 없다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요!"

    사용자: "내 이름이 뭐였지?"
    AI: "죄송해요, 모르겠습니다"
```
> Memory가 있다면:
```text
    사용자: "내 이름은 김철수야"
    AI: "안녕하세요 김철수님!"

    사용자: "내 이름이 뭐였지?"
    AI: "김철수님이라고 하셨어요!"
```


# 암호화된 SQLite Memory 예시 (EncryptedSerializer)
**보안이 중요한 대화를 암호화해서 저장하는 챗봇**

실제 서비스에서는 대화 내용을 안전하게 보관해야 해요. SQLite 데이터베이스에 암호화해서 저장하는 방법입니다!


In [ ]:
# 필요한 라이브러리 import
from dotenv import load_dotenv

# 환경변수 로드
load_dotenv()

### 1. AI 모델 설정

In [28]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

### 2. 챗봇 노드 정의

In [29]:
from langgraph.graph import MessagesState

def secure_chat_node(state: MessagesState):
    """보안이 적용된 챗봇 노드"""
    response = llm.invoke(state["messages"])
    return {"messages": [response]}

### 4. 그래프 생성

In [30]:
from langgraph.graph import StateGraph, MessagesState, START, END

secure_graph = StateGraph(MessagesState)
secure_graph.add_node("chat", secure_chat_node)
secure_graph.add_edge(START, "chat")
secure_graph.add_edge("chat", END)

### 5. 암호화 키 설정 (실제 운영에서는 환경변수로 관리!)

In [31]:
import hashlib 
import os

def aes_key(seed_string:str):
    return hashlib.sha256(str(seed_string).encode()).hexdigest()[:32]

seed = "Development" # 개발인 경우 
# 운영인 경우 
if os.getenv("ENV") == "Production":
    seed = os.getenv("SECRET_SEED", "your-secret-seed-here")

os.environ["LANGGRAPH_AES_KEY"] = aes_key(seed)
print("암호화 키가 설정되었습니다!")
print("실제 운영에서는 환경변수나 보안 저장소에서 키를 가져오세요!")

암호화 키가 설정되었습니다!
실제 운영에서는 환경변수나 보안 저장소에서 키를 가져오세요!


### 6. 암호화된 SQLite 저장소 설정

In [32]:
import sqlite3
from langgraph.checkpoint.serde.encrypted import EncryptedSerializer
from langgraph.checkpoint.sqlite import SqliteSaver

# check_same_thread=False 옵션으로 멀티스레드 환경에서 사용 가능하게 설정
def get_db_connection():
    return sqlite3.connect("secure_memory.db", check_same_thread=False)  # SQLite 데이터베이스

serde = EncryptedSerializer.from_pycryptodome_aes()  # 암호화 직렬화기
checkpointer = SqliteSaver(get_db_connection(), serde=serde)  # 암호화된 저장소

### 7. 암호화된 Memory와 함께 컴파일

In [33]:
secure_app = secure_graph.compile(checkpointer=checkpointer)

print("암호화된 SQLite Memory 챗봇이 준비되었습니다!")
print("대화 내용이 'secure_memory.db' 파일에 암호화되어 저장됩니다!")

암호화된 SQLite Memory 챗봇이 준비되었습니다!
대화 내용이 'secure_memory.db' 파일에 암호화되어 저장됩니다!


### 8. 테스트 

In [34]:
import uuid

# 중요: 같은 thread_id를 사용해야 대화가 연결됩니다!
memory_id = str(uuid.uuid4())
# 암호화된 Memory 테스트
config_secure = {"configurable": {"thread_id": memory_id}}

print("=== 암호화된 SQLite Memory 챗봇 테스트 ===")
print("중요한 개인정보를 포함한 대화를 테스트해보겠습니다!\\n")
print(f"config_secure: {config_secure}")

=== 암호화된 SQLite Memory 챗봇 테스트 ===
중요한 개인정보를 포함한 대화를 테스트해보겠습니다!\n
config_secure: {'configurable': {'thread_id': 'ef751c5c-ea70-4afc-a79b-8cbbf40605c3'}}


In [35]:
from langchain_core.messages import HumanMessage

# 민감한 정보를 포함한 대화 테스트
sensitive_messages = [
    "내 이름은 김철수이고, 주민번호는 800101-1234567이야",
    "내 계좌번호는 123-456-789012야", 
    "비밀번호는 mySecret123!이야",
    "내 개인정보가 뭐였지?",
    "계좌번호를 다시 알려줘"
]

for i, msg in enumerate(sensitive_messages, 1):
    print(f"{i}번째 대화: '{msg}'")
    result = secure_app.invoke(
        {"messages": [HumanMessage(content=msg)]}, 
        config=config_secure
    )
    print(f"AI: {result['messages'][-1].content}")
    print("-" * 50)

print("\\n결과: 모든 대화가 암호화되어 안전하게 저장되었습니다!")
print("데이터베이스를 직접 열어봐도 암호화된 내용만 보입니다!")

1번째 대화: '내 이름은 김철수이고, 주민번호는 800101-1234567이야'
AI: 죄송하지만, 개인 정보를 요청하거나 저장할 수 없습니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
2번째 대화: '내 계좌번호는 123-456-789012야'
AI: 죄송하지만, 개인적인 금융 정보나 민감한 정보를 공유하실 수 없습니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
3번째 대화: '비밀번호는 mySecret123!이야'
AI: 죄송하지만, 비밀번호와 같은 민감한 정보를 공유하실 수 없습니다. 보안을 위해 개인 정보를 안전하게 보호하는 것이 중요합니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
4번째 대화: '내 개인정보가 뭐였지?'
AI: 죄송하지만, 저는 개인 정보를 저장하거나 기억할 수 없습니다. 또한, 개인정보 보호를 위해 그러한 정보를 요청하거나 처리하지 않습니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
5번째 대화: '계좌번호를 다시 알려줘'
AI: 죄송하지만, 개인적인 정보나 계좌번호와 같은 민감한 정보를 저장하거나 제공할 수 없습니다. 보안과 개인정보 보호를 위해 이러한 정보를 안전하게 관리하는 것이 중요합니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!
--------------------------------------------------
\n결과: 모든 대화가 암호화되어 안전하게 저장되었습니다!
데이터베이스를 직접 열어봐도 암호화된 내용만 보입니다!


### 9. 데이터베이스 내용 확인 (암호화 확인)

#### 보안 장점 
- 대화 내용이 암호화되어 저장 
- 키 없이는 데이터 복호화 불가능 
- SQLite 파일을 직접 열어도 내용 확인 불가 
- 실제 서비스에 적합한 보안 수준 

In [ ]:
print("\\n=== 데이터베이스 암호화 확인 ===")

# SQLite 데이터베이스에서 실제 저장된 데이터 확인
cursor = get_db_connection().cursor()

try:
    # 테이블 구조 확인
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tables = cursor.fetchall()
    print(f"생성된 테이블들: {tables}")
    
    # 실제 데이터 일부 확인 (암호화된 상태)
    cursor.execute("SELECT * FROM checkpoints LIMIT 1")
    encrypted_data = cursor.fetchone()
    
    if encrypted_data:
        print("\\n데이터베이스에 저장된 실제 데이터 (일부):")
        print(f"Thread ID: {encrypted_data[0] if encrypted_data[0] else 'N/A'}")
        print(f"암호화된 데이터 (처음 100자): {str(encrypted_data)[:100]}...")
        print("\\n보시다시피 실제 대화 내용은 암호화되어 읽을 수 없습니다!")
    else:
        print("아직 저장된 데이터가 없습니다.")
        
except Exception as e:
    print(f"데이터베이스 확인 중 오류: {e}")


\n=== 데이터베이스 암호화 확인 ===
생성된 테이블들: [('checkpoints',), ('writes',)]
\n데이터베이스에 저장된 실제 데이터 (일부):
Thread ID: ef751c5c-ea70-4afc-a79b-8cbbf40605c3
암호화된 데이터 (처음 100자): ('ef751c5c-ea70-4afc-a79b-8cbbf40605c3', '', '1f07e6bc-e171-6088-bfff-5452d229018d', None, 'msgpack+...
\n보시다시피 실제 대화 내용은 암호화되어 읽을 수 없습니다!
\n보안 장점:
1. 대화 내용이 암호화되어 저장
2. 키 없이는 데이터 복호화 불가능
3. SQLite 파일을 직접 열어도 내용 확인 불가
4. 실제 서비스에 적합한 보안 수준


### EncryptedSerializer 사용 시 주의사항

#### 필수 설치 패키지
```bash
pip install pycryptodome  # 암호화를 위해 필요
```

#### 암호화 키 관리
```python
# 나쁜 예시 - 코드에 직접 하드코딩
os.environ["LANGGRAPH_AES_KEY"] = "hardcoded-key"

# 좋은 예시 - 환경변수나 보안 저장소 사용
import os
key = os.getenv("LANGGRAPH_AES_KEY")  # 환경변수에서 가져오기
```

#### 실제 운영 환경에서의 권장사항
1. **환경변수로 키 관리**: 코드에 키를 하드코딩하지 말 것
2. **키 회전 정책**: 정기적으로 암호화 키 변경
3. **백업 정책**: 암호화된 데이터베이스 백업 계획
4. **접근 권한**: 데이터베이스 파일 접근 권한 제한


# 업데이트된 Memory 종류별 정리

| Memory 종류 | 장점 | 단점 | 언제 사용할까? | 보안 수준 |
|------------|------|------|----------------|-----------|
| **Memory 없음** | 빠름, 단순함 | 대화 기억 안함 | 일회성 질문답변 | 보통 |
| **MemorySaver** | 모든 대화 기억 | 메모리 많이 사용 | 짧은 대화, 모든 내용 중요 | 보통 |
| **WindowMemory** | 적당한 메모리 사용 | 오래된 내용 잊음 | 긴 대화, 최근 내용만 중요 | 보통 |
| **SummaryMemory** | 효율적, 핵심 유지 | 요약 과정 필요 | 긴 대화, 핵심 정보 유지 | 보통 |
| **EncryptedSQLite** | 영구저장, 보안 | 설정 복잡, 성능 오버헤드 | 운영 서비스, 개인정보 포함 | **높음** |

### 실제 사용 시나리오별 추천

#### 개발/테스트 환경
- **MemorySaver**: 간단한 테스트용
- **WindowMemory**: 성능 테스트용

#### 운영 환경  
- **EncryptedSQLite**: 개인정보/민감정보 포함 서비스
- **SummaryMemory + SQLite**: 일반 서비스

#### 모바일/임베디드
- **WindowMemory**: 메모리 제약 환경
- **InMemory**: 일시적 사용
